In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

from spicy_snow.retrieval import retrieval_from_parameters

In [2]:
in_dir = Path('/Users/rdcrlzh1/Documents/spicy-analysis/data/SnowEx-Data')
dss = {fp.stem: xr.open_dataset(fp) for fp in in_dir.glob('*.nc')}


In [64]:
res = pd.DataFrame()
i = 0
for stem, ds in sorted(dss.items()):
    for orbit in np.unique(ds.relative_orbit.data):
        orbit_ds = ds.sel(time = ds.relative_orbit == orbit)
            
            # print(ts.data)
            # print(ds.sel(time = ts).flight_dir.data)
            # print(ds.sel(time = ts).relative_orbit.data)
        # print(stem, orbit)
        # print([pd.to_datetime(t).strftime('%Y-%m-%d') for t in orbit_ds.time.data])
        for ts in [pd.to_datetime(t).strftime('%Y-%m-%d') for t in orbit_ds.time.data]:
            res.loc[i, "site"] = stem.split('_')[0]
            res.loc[i, "satellite"] = orbit_ds.sel(time = ts).platform.data[0]
            res.loc[i, 'orbit'] = orbit
            res.loc[i, 'direction'] = orbit_ds.sel(time = ts).flight_dir.data[0]
            res.loc[i, "time"] = ts
            i += 1

/var/folders/rf/mzlwcxzn6w700njgf1fwn7ch0000gs/T/ipykernel_48911/645548717.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Banner' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  res.loc[i, "site"] = stem.split('_')[0]
/var/folders/rf/mzlwcxzn6w700njgf1fwn7ch0000gs/T/ipykernel_48911/645548717.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'S1B' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  res.loc[i, "satellite"] = orbit_ds.sel(time = ts).platform.data[0]
/var/folders/rf/mzlwcxzn6w700njgf1fwn7ch0000gs/T/ipykernel_48911/645548717.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'ascending' has dtype incompatible with float64, please explicitly cast to a compatible dtype first

In [66]:
res.to_csv('time.csv')

In [122]:
res

,site,satellite,orbit,direction,time
0,Banner,S1B,20.0,ascending,2019-08-12
1,Banner,S1B,20.0,ascending,2019-08-24
2,Banner,S1B,20.0,ascending,2019-09-05
3,Banner,S1B,20.0,ascending,2019-09-17
4,Banner,S1B,20.0,ascending,2019-09-29
...,...,...,...,...,...
602,Mores,S1B,93.0,ascending,2021-03-03
603,Mores,S1A,93.0,ascending,2021-03-09
604,Mores,S1B,93.0,ascending,2021-03-15
605,Mores,S1A,93.0,ascending,2021-03-21


In [121]:
for site in res.site.unique():
    site_df = res.loc[res.site == site, :]
    for satellite in site_df.satellite.unique():
        site_sat_df = site_df.loc[site_df.satellite == satellite, :]
        for direction in site_sat_df.direction.unique():
            site_sat_direction_df = site_sat_df.loc[site_sat_df.direction == direction, :]

            for orbit in site_sat_direction_df.orbit.unique():
                orbit_df = site_sat_direction_df.loc[site_sat_direction_df.orbit == orbit, :]
            print('\\hline')
            print(f'{site} & {satellite} & {direction} & {orbit} &', end = '')  
            start = 0          
            chunk_size = 4
            for i in chunker(orbit_df, chunk_size):
                if start != 0:
                    print(' & & & &', end= '')
                else: start = 1
                for t in i.time.values:
                    if t == i.time.values[-1]:
                        print(f'{t}\\\\')
                    else:
                        print(f' {t},', end = '')
        # print(site_df)
print(f'\\hline')

\hline
Banner & S1B & ascending & 93.0 & 2019-08-05, 2019-08-17, 2019-08-29,2019-09-10\\
 & & & & 2019-09-22, 2020-01-20, 2020-02-01,2020-02-13\\
 & & & & 2020-02-25, 2020-08-11, 2020-08-23,2020-09-04\\
 & & & & 2020-09-16, 2020-09-28, 2020-10-10,2020-10-22\\
 & & & & 2020-11-03, 2020-11-15, 2020-11-27,2020-12-09\\
 & & & & 2020-12-21, 2021-01-02, 2021-01-14,2021-01-26\\
 & & & & 2021-02-07, 2021-02-19, 2021-03-03,2021-03-15\\
 & & & &2021-03-27\\
\hline
Banner & S1B & descending & 71.0 & 2019-11-07, 2019-11-19, 2019-12-01,2019-12-13\\
 & & & & 2019-12-25, 2020-01-06, 2020-01-18,2020-01-30\\
 & & & & 2020-02-11, 2020-02-23, 2020-08-09,2020-08-21\\
 & & & & 2020-09-02, 2020-09-14, 2020-09-26,2020-10-08\\
 & & & & 2020-10-20, 2020-11-01, 2020-11-13,2020-11-25\\
 & & & & 2020-12-07, 2020-12-19, 2020-12-31,2021-01-12\\
 & & & & 2021-01-24, 2021-02-17, 2021-03-01,2021-03-13\\
 & & & &2021-03-25\\
\hline
Banner & S1A & descending & 71.0 & 2019-09-26, 2019-11-01, 2019-11-13,2019-11-25\\
 & & 

In [94]:
def chunker(seq, size):
    for pos in range(0, len(seq), size):
        yield seq.iloc[pos:pos + size] 
